# Get curb heights from point cloud


### NOTE: First run the ground_and_road_fusion.ipynb notebook, otherwise this notebook does not work

In [ ]:
import set_path

import upcp.scrapers.ams_bgt_scraper as ams_bgt_scraper
import upcp.utils.las_utils as las_utils
import upcp.utils.bgt_utils as bgt_utils

from upcp.labels import Labels
from upc_sw.las_utils_extra import read_las
from upc_sw.sidewalk_filter import *

In [ ]:
import os
import numpy as np
import pandas as pd
import seaborn as sns

import matplotlib.pyplot as plt
from shapely.geometry import LineString, Point, MultiPoint, MultiLineString, Polygon, MultiPolygon 
from shapely.ops import unary_union, split, snap

import geopandas as gpd
import folium
from tqdm import tqdm
from geopandas import GeoDataFrame
import branca.colormap as cm

import functions as f

import requests
import statistics

import warnings
from shapely.errors import ShapelyDeprecationWarning
warnings.filterwarnings("ignore", category=ShapelyDeprecationWarning) 

import config_azure as cf

In [ ]:
CRS = 'epsg:28992'

In [ ]:
def create_mask(points, polygons):
    label_mask = np.zeros(len(points), dtype=bool)

    # Already labelled ground points can be labelled as road.
    mask = (labels == Labels.GROUND) | (labels == Labels.ROAD)
    mask_ids = np.where(mask)[0]

    road_mask = np.zeros((len(mask_ids),), dtype=bool)
    for polygon in polygons:
        clip_mask = clip_utils.poly_clip(points[mask, :], polygon)
        road_mask = road_mask | clip_mask

    label_mask[mask_ids[road_mask]] = True

    return label_mask

def create_mask2(points, polygons):
    label_mask = np.zeros(len(points), dtype=bool)

    for polygon in polygons:
        clip_mask = clip_utils.poly_clip(points, polygon)
        label_mask = label_mask | clip_mask

    return label_mask

In [ ]:
# Functions for method 1

def bbox_to_polygon(bbox):
    return Polygon([bbox[0],(bbox[0][0],bbox[1][1]), bbox[1],(bbox[1][0],bbox[0][1])])

def calculate_curb_height_median(points, labels, segment_polygon, nr_points): # calculate median over all points
    curb_height = np.nan
    available_points = True

    label_mask = create_mask(points, [segment_polygon,]) # TODO
    points_in_segment = points[label_mask]
    labels_in_segment = labels[label_mask]
    
    z_values_road = points_in_segment[labels_in_segment == Labels.ROAD][:, -1]

    if len(z_values_road) > min_nr_points:
        z_values_road.sort()
        road_height = statistics.median(z_values_road)
    else:
        available_points = False

    z_values_sidewalk = points_in_segment[labels_in_segment == Labels.GROUND][:, -1]
    if len(z_values_sidewalk) > min_nr_points:
        z_values_sidewalk.sort()
        sidewalk_height = statistics.median(z_values_sidewalk)
    else:
        available_points = False

    if available_points:
        curb_height = sidewalk_height - road_height
    
    return curb_height, available_points

def get_height_color(curb_height, available_points, min_h):
    if not available_points:
        color = 'black'
        return color
    
    if curb_height < min_h:
        color = 'green'
    else:
        color = 'orange'
    return color

def get_points_on_line(line, distance_delta):
    # generate the equidistant points
    distances = np.arange(0, line.length, distance_delta)
    points = MultiPoint([line.interpolate(distance) for distance in distances] + [(line.coords[0], line.coords[-2])]) # line.boundary[1]
    return points[:-1] # Exclude last (duplicate) point

def split_line_by_point(line, point, tolerance: float=1): # TO DO Tolerence is belangrijk, misschien hier nog mee tweaken
    return split(snap(line, point, tolerance), point)

def plot_line(ax, ob):
    x, y = ob.xy
    ax.plot(x, y, linewidth=2, color='black')

def plot_poly(ax, poly, color='C0'):
    x,y = poly.exterior.xy
    plt.fill(x, y, color=color, alpha=0.3)
    plt.plot(x, y, color=color)

# 0. Prepare

In [ ]:
# # Pull polygon of area for which to gather curb segments
os.system('sudo blobfuse /home/azureuser/cloudfiles/code/blobfuse/sidewalk --tmp-path=/mnt/resource/blobfusetmp --config-file=/home/azureuser/cloudfiles/code/blobfuse/fuse_connection_sidewalk.cfg -o attr_timeout=3600 -o entry_timeout=3600 -o negative_timeout=3600 -o allow_other -o nonempty')

# Import areas
CRS = 'epsg:28992'
df_areas = gpd.read_file(cf.output_pilot_area)
polygon = df_areas.to_crs(crs=CRS).unary_union

## Select possible tiles

In [ ]:
# Mount ovl container to access point clouds and set folders
os.system('sudo blobfuse /home/azureuser/cloudfiles/code/blobfuse/ovl --tmp-path=/mnt/resource/blobfusetmp --config-file=/home/azureuser/cloudfiles/code/blobfuse/fuse_connection_ovl.cfg -o attr_timeout=3600 -o entry_timeout=3600 -o negative_timeout=3600 -o allow_other -o nonempty')
base_folder = "/home/azureuser/cloudfiles/code/blobfuse/ovl/"
base_folder_bgt = base_folder + 'bgt/bgt_roads/'
base_folder_images = '../output/media/road_curb_segments/'
in_folder_point_clouds = base_folder + 'pointcloud/Labeled/ground_and_road/'

if not os.path.isdir(base_folder_images):
    os.makedirs(base_folder_images)

In [ ]:
# Collect all tiles in polygon area
all_pc_filenames = []
for path, subdirs, files in os.walk(in_folder_point_clouds):
    for name in files:
            if '.laz' in name:
                all_pc_filenames.append(os.path.join(path, name))
all_pc_filenames = np.array(all_pc_filenames)
all_pc_tilecodes = np.array([las_utils.get_tilecode_from_filename(filename) for filename in all_pc_filenames])
all_pc_bboxes = np.array([las_utils.get_bbox_from_tile_code(tilecode) for tilecode in all_pc_tilecodes])

all_pc_polygons = [las_utils.get_polygon_from_tile_code(tilecode) for tilecode in all_pc_tilecodes]
gdf_pc_polygons = gpd.GeoDataFrame(geometry=all_pc_polygons)

gdf = gdf_pc_polygons.intersection(polygon)
pc_idxs_in_area_polygon = gdf[~gdf.is_empty].index.to_list()

# filenames, tilecodes and bounding boxes of point clouds in selected area
pc_filenames = all_pc_filenames[pc_idxs_in_area_polygon]
pc_tilecodes = all_pc_tilecodes[pc_idxs_in_area_polygon]
pc_bboxes = all_pc_bboxes[pc_idxs_in_area_polygon]

sns.set()
ax = gdf_pc_polygons.iloc[pc_idxs_in_area_polygon].boundary.plot()
plt.title('Point clouds in selected area')
plt.show()

## Generate dictionary with point cloud tile information
This contains:
tilecode, filename, point cloud tile polygon, sidewalk polygons, parkeervlak polygons

In [ ]:
tiles_bgt_dict = {}
for tilecode, filename, bbox in zip(pc_tilecodes, pc_filenames, pc_bboxes):
    bgt_data_file = base_folder_bgt + '{}.csv'.format(tilecode)

    ((x_min, y_max), (x_max, y_min)) = bbox

    # Create reader for BGT sidewalk part polygons.
    bgt_sidewalk_reader = bgt_utils.BGTPolyReader(bgt_file=bgt_data_file)

    sidewalk_polygons = bgt_sidewalk_reader.filter_tile(
                                tilecode,
                                padding=0, offset=0,
                                merge=False)

    parkeervlak_polygons = bgt_sidewalk_reader.filter_tile(
                                tilecode,
                                padding=0, offset=0,
                                bgt_types=['parkeervlak',],
                                merge=False)

    tiles_bgt_dict[tilecode] = {'filename': filename, 
                                'bbox': bbox, 
                                'tile_polygon': bbox_to_polygon(bbox),
                                'sw_polygons': sidewalk_polygons, 
                                'pkv_polygons': parkeervlak_polygons,
                                'lines_in_tile': None,
                                'potential_crossing_lines': None}

# 1. First method - heights

## Add all lines and potential crossing lines to point cloud information dictionary
We split line between sidewalk and road in segments. Next, we create overlapping polygons of these line segments with given buffer along the line.

In [ ]:
# Loop over point cloud and generate curb height images
for tilecode, values in tiles_bgt_dict.items():
    sidewalk_polygons = tiles_bgt_dict[tilecode]['sw_polygons']
    parkeervlak_polygons = tiles_bgt_dict[tilecode]['pkv_polygons']

    # Check if point cloud tile holds sidewalk polygons
    if sidewalk_polygons:
        tile_polygon = tiles_bgt_dict[tilecode]['tile_polygon']
        sidewalks_merged = unary_union(sidewalk_polygons)
        road_in_tile = sidewalks_merged.intersection(tile_polygon)
        
        boundary = road_in_tile.boundary
        lines_in_tile = boundary.intersection(tile_polygon.buffer(-0.5))

        # Remove curbs next to parking, these are not accessible
        potential_crossing_lines = lines_in_tile
        for parkeer in parkeervlak_polygons:
            potential_crossing_lines = potential_crossing_lines.difference(parkeer)

        # Add lines to dictionary
        tiles_bgt_dict[tilecode]['lines_in_tile'] = lines_in_tile
        tiles_bgt_dict[tilecode]['potential_crossing_lines'] = potential_crossing_lines

### Set parameters for curb detection

In [ ]:
## Tweak parameters

# Cut offs for height categories
min_h = 0.04 # 0.07
#mid_h = 0.15  # 0.15

# Width of the buffer (to both sides)
buffer_w = 0.15 # 0.5

# Amount of points to take the median height from 
nr_points = 200 # 500, 1000

# Minimum amount of points within polygon to continue
min_nr_points = nr_points # 1000

# length of polygons
distance_delta = 1 # 2

### Plot possible crossing lines on map and save as csv

In [ ]:
# Load curb heights csv file
curb_height_df = pd.DataFrame(columns=['line_segm', 'line_segm_polygon', 'overarching_line_segm','curb_height'])

# Calculate curb heights and save in csv
for tilecode, values in tqdm(tiles_bgt_dict.items()):
    potential_crossing_lines = tiles_bgt_dict[tilecode]['potential_crossing_lines']

    # if potential_crossing_lines and not os.path.isfile(base_folder_images + '{}_height.png'.format(tilecode)):
    if potential_crossing_lines:
        lines_in_tile = tiles_bgt_dict[tilecode]['lines_in_tile']
        potential_crossing_lines = tiles_bgt_dict[tilecode]['potential_crossing_lines']
        filename = tiles_bgt_dict[tilecode]['filename']
        try:
            points, labels = read_las(filename)

            if potential_crossing_lines.type == 'LineString':
                potential_crossing_lines = [potential_crossing_lines]

            for line in potential_crossing_lines:

                points_on_line = get_points_on_line(line, distance_delta)
                result = split_line_by_point(line, points_on_line)

                for line_segm in result:
                    poly = line_segm.buffer(0.5, single_sided=True)
                    poly2 = line_segm.buffer(-0.5, single_sided=True)
                    poly_union = unary_union([poly, poly2])

                    if line_segm.length < (distance_delta/2):
                        continue

                    # calc height difference planes of sidewalk and road using point cloud
                    if poly_union.type == 'MultiPolygon':
                        curb_height, available_points = 0, False
                    else:
                        curb_height, available_points = calculate_curb_height_median(points, labels, poly_union, nr_points)
                        height_color = get_height_color(curb_height, available_points, min_h)
                    
                    # add curb information to dataframe
                    new_row = [line_segm, poly_union, line, curb_height]
                    curb_height_df.loc[len(curb_height_df)] = new_row
        except:
            continue

### export curb heights as csv and create html map curb heights and

In [ ]:
curb_height_df['geometry'] = curb_height_df['line_segm']
gdf = gpd.GeoDataFrame(curb_height_df, crs=CRS)
gdf = gdf.drop(columns=['line_segm', 'line_segm_polygon', 'overarching_line_segm'])
gdf['tilecode'] = np.floor(gdf.centroid.x/50).astype(int).astype(str) + '_' + np.floor(gdf.centroid.y/50).astype(int).astype(str)
gdf.fillna(100, inplace=True)

In [ ]:
# Create tooltip for feature representation on map
def gen_tooltip(fields, aliases):

    tooltip = folium.GeoJsonTooltip(
        fields=fields,
        aliases=aliases,
        localize=True,
        sticky=False,
        labels=True,
        style="""
            background-color: #F0EFEF;
            border: 2px solid black;
            border-radius: 3px;
            box-shadow: 3px;
        """,
        max_width=800,
    )
    return tooltip

# set True for satelite background, False for regular background
satelite = False

# Set folium map background
if satelite == True:
    network_color = 'white'
    tile = folium.TileLayer(
                tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
                attr = 'Esri',
                name = 'Esri Satellite',
                overlay = False,
                control = True)
else:
    tile = 'openstreetmap'
    network_color = 'black'

# Create Folium map
map = folium.Map(
    location=[52.389164, 4.908453], tiles=tile,
    min_zoom=10, max_zoom=25, zoom_start=15,
    zoom_control=True, control_scale=True, control=False
    )

# Add colormap
cmp = cm.linear.RdYlGn_11.colors
cmp = list(reversed(cmp))
colormap = cm.LinearColormap(colors=cmp, vmin=0, vmax=0.2, caption='Curb height (m)')
colormap.add_to(map)

# Add curb height features
json = gdf.to_crs(crs='EPSG:4326').to_json()
feature_names = gdf.columns.tolist()
feature_names.remove('geometry')
tooltip = gen_tooltip(feature_names, feature_names)
folium.GeoJson(json, style_function=lambda feature: {"color": colormap(feature["properties"]['curb_height']) if feature['properties']['curb_height'] != 100 else 'black', 
            "weight": 7 if feature["properties"]['curb_height'] != 100 else 2}, tooltip=tooltip).add_to(map)

In [ ]:
# Change directory 
os.system('sudo blobfuse /home/azureuser/cloudfiles/code/blobfuse/sidewalk --tmp-path=/mnt/resource/blobfusetmp --config-file=/home/azureuser/cloudfiles/code/blobfuse/fuse_connection_sidewalk.cfg -o attr_timeout=3600 -o entry_timeout=3600 -o negative_timeout=3600 -o allow_other -o nonempty')

# Save curb height data as csv
curb_height_df.to_csv(cf.out_folder + '/curb_heights/curbs_and_heights.csv')

# Save network with curb height features as html map
if satelite == True:
    map.save(cf.out_folder + '/curb_heights/curb_height_map_gradients_satelite.html')
else:
    map.save(cf.out_folder + '/curb_heights/curb_height_map_gradients.html')